In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.tools import PointDrawTool
from bokeh.io import push_notebook
from IPython.display import display

output_notebook()

# Datos iniciales
source = ColumnDataSource(data=dict(x=[1, 2, 3, 4], y=[3, 7, 5, 6]))

# Crear figura
p = figure(title="Arrastra los puntos para editar",
           x_axis_label='X', y_axis_label='Y',
           width=700, height=400,
           tools="reset,pan,wheel_zoom,save")

# Dibujar puntos y líneas
p.line('x', 'y', source=source, line_width=2, color="blue")
renderer = p.circle('x', 'y', source=source, size=10, color="red")

# Herramienta para arrastrar puntos
draw_tool = PointDrawTool(renderers=[renderer], add=False)
p.add_tools(draw_tool)
p.toolbar.active_drag = draw_tool

# Mostrar en notebook con interacción en vivo
handle = show(p, notebook_handle=True)


In [1]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LassoSelectTool, CustomJS
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.layouts import column
from bokeh.io import push_notebook
from IPython.display import display

output_notebook()

# Fuente de datos
source = ColumnDataSource(data=dict(x=[1, 2, 3, 4, 5], y=[3, 6, 2, 7, 5]))

# Crear figura
p = figure(title="Herramienta de lazo activa",
           x_axis_label='X', y_axis_label='Y',
           tools="lasso_select,reset,pan,wheel_zoom",
           width=700, height=400)

# Dibujar puntos y línea
p.line('x', 'y', source=source, line_width=2, color="gray", alpha=0.5)
p.circle('x', 'y', source=source, size=10, color='green', selection_color='orange')

# Mostrar tabla de selección
columns = [TableColumn(field="x", title="X"),
           TableColumn(field="y", title="Y")]
table = DataTable(source=source, columns=columns, width=400, height=200)

# Mostrar
layout = column(p, table)
show(layout, notebook_handle=True)


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, TableColumn, DataTable
from bokeh.layouts import column
from bokeh.io import push_notebook
from IPython.display import display
import ipywidgets as widgets
import numpy as np

output_notebook()

# Datos de ejemplo
x = [1, 2, 3, 4, 5, 6, 7]
y = [4, 2, 5, 8, 4, 6, 9]
source = ColumnDataSource(data=dict(x=x, y=y))

# Crear figura
p = figure(title="Interpolación lineal de puntos seleccionados",
           tools="lasso_select,reset,pan,wheel_zoom", width=700, height=400)
p.circle('x', 'y', size=10, source=source, color='green', selection_color="red")
p.line('x', 'y', source=source, line_dash='dashed', color="gray", alpha=0.5)

# Tabla
columns = [TableColumn(field="x", title="X"),
           TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=200)

# Botón de interpolar
boton_interpolar = widgets.Button(description="Interpolar seleccionados", button_style="success")

def interpolar_linea(b):
    datos = source.data
    seleccionados = sorted(source.selected.indices)  # ordenados por índice

    if len(seleccionados) < 2:
        print("⚠️ Selecciona al menos dos puntos para interpolar.")
        return

    # Extraer extremos
    i0 = seleccionados[0]
    i1 = seleccionados[-1]

    x0, y0 = datos['x'][i0], datos['y'][i0]
    x1, y1 = datos['x'][i1], datos['y'][i1]

    # Interpolar entre los extremos
    for i in seleccionados:
        if i == i0:
            datos['y'][i] = y0
        elif i == i1:
            datos['y'][i] = y1
        else:
            xi = datos['x'][i]
            # y = y0 + pendiente * (x - x0)
            pendiente = (y1 - y0) / (x1 - x0) if (x1 - x0) != 0 else 0
            datos['y'][i] = y0 + pendiente * (xi - x0)

    # Actualizar fuente
    source.data = dict(x=datos['x'], y=datos['y'])
    push_notebook()

# Conectar botón
boton_interpolar.on_click(interpolar_linea)

# Mostrar interfaz
display(widgets.VBox([boton_interpolar]))
show(column(p, tabla), notebook_handle=True)


⚠️ Selecciona al menos dos puntos para interpolar.
⚠️ Selecciona al menos dos puntos para interpolar.
⚠️ Selecciona al menos dos puntos para interpolar.
⚠️ Selecciona al menos dos puntos para interpolar.
⚠️ Selecciona al menos dos puntos para interpolar.
⚠️ Selecciona al menos dos puntos para interpolar.
⚠️ Selecciona al menos dos puntos para interpolar.


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row

import numpy as np
output_notebook()

# Datos de ejemplo
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"]*len(x)))

# Figura
p = figure(title="Gráfica interactiva con puntos", tools="tap,box_select,lasso_select,reset", width=800, height=400)

# Solo puntos, con color dinámico
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)

# Dropdown (por ahora solo con 'puntos' por limitación de líneas)
selector = Select(title="Tipo de gráfico:", value="puntos", options=["puntos"])
# (Nota: no hacemos cambio aquí porque no hay líneas)

# Cambiar color al seleccionar puntos
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;

    // Restaurar todos a azul
    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }

    // Colorear seleccionados de rojo
    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }

    source.change.emit();
"""))

# Tabla de selección
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# Mostrar
layout = column(selector, row(p, tabla))
show(layout)


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
import numpy as np

output_notebook()

# Datos de ejemplo
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"]*len(x)))

# Crear figura
p = figure(title="Gráfica interactiva con distintos tipos", tools="tap,box_select,lasso_select,reset", width=800, height=400)

# Glyphs: puntos y línea
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# Selector de tipo de gráfica
selector = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])

# Callback JS para cambiar visibilidad según selección
callback = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;

    if (tipo === "puntos") {
        puntos.visible = true;
        linea.visible = false;
    } else if (tipo === "línea") {
        puntos.visible = false;
        linea.visible = true;
    } else if (tipo === "mixto") {
        puntos.visible = true;
        linea.visible = true;
    }
""")

selector.js_on_change('value', callback)

# Cambiar color al seleccionar puntos (igual que antes)
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;

    // Restaurar todos a azul
    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }

    // Colorear seleccionados de rojo
    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }

    source.change.emit();
"""))

# Tabla para mostrar datos
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# Layout y mostrar todo
layout = column(selector, row(p, tabla))
show(layout)


/tmp/ipython-input-1175301291.py:17: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.



In [2]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Datos
df = pd.DataFrame({
    'x': range(1, 21),
    'y': [2, 5, 3, 8, 7, 6, 12, 10, 15, 13, 14, 11, 9, 7, 5, 8, 12, 14, 13, 10],
    'grupo': ['A']*10 + ['B']*10
})

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='color_selector',
        options=[{'label': g, 'value': g} for g in df['grupo'].unique()],
        value='A',
        clearable=False
    ),
    dcc.Graph(id='scatter_plot')
])

@app.callback(
    Output('scatter_plot', 'figure'),
    Input('color_selector', 'value')
)
def update_graph(selected_group):
    filtered_df = df[df['grupo'] == selected_group]
    fig = px.scatter(filtered_df, x='x', y='y', color='grupo', title=f"Grupo {selected_group}")
    fig.update_traces(marker_size=15)
    return fig

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'dash'

In [4]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, PointDrawTool, Select
from bokeh.layouts import column, row
import numpy as np

output_notebook()

# Datos iniciales
x = np.linspace(0, 10, 50)
y = np.sin(x)

source_all = ColumnDataSource(data=dict(x=x, y=y))

# Fuente para línea (no seleccionados o todo según modo)
source_line = ColumnDataSource(data=dict(x=x, y=y))

# Fuente para puntos (fijos o seleccionados)
source_points = ColumnDataSource(data=dict(x=x, y=y, index=list(range(len(x)))))

# Crear figura
p = figure(title="Modos: puntos, línea, mixto y editable",
           tools="lasso_select,box_select,reset,pan,wheel_zoom",
           width=800, height=400)

# Glyphs
line_renderer = p.line('x', 'y', source=source_line, line_width=2, color='blue')
points_renderer = p.scatter('x', 'y', source=source_points, size=8, color='blue')

# Herramienta para editar puntos (solo para modo editable)
draw_tool = PointDrawTool(renderers=[points_renderer], add=False)
p.add_tools(draw_tool)

# Selector de modo
select_mode = Select(title="Modo de visualización:", value="mixto",
                     options=["puntos", "línea", "mixto", "editable"])

# JS callback para cambiar visibilidad y comportamiento según modo
callback_mode = CustomJS(args=dict(
    source_all=source_all, source_line=source_line, source_points=source_points,
    line_renderer=line_renderer, points_renderer=points_renderer, draw_tool=draw_tool, p=p
), code="""
    const modo = cb_obj.value;

    if (modo === "puntos") {
        line_renderer.visible = false;
        points_renderer.visible = true;
        points_renderer.glyph.fill_color = 'blue';
        p.toolbar.active_drag = null;  // desactivar drag
    } else if (modo === "línea") {
        line_renderer.visible = true;
        points_renderer.visible = false;
        p.toolbar.active_drag = null;
    } else if (modo === "mixto") {
        line_renderer.visible = true;
        points_renderer.visible = true;
        points_renderer.glyph.fill_color = 'blue';
        p.toolbar.active_drag = null;
    } else if (modo === "editable") {
        line_renderer.visible = true;
        points_renderer.visible = true;
        points_renderer.glyph.fill_color = 'red';  // editable puntos rojos
        p.toolbar.active_drag = draw_tool;  // activar mover puntos
    }
""")

select_mode.js_on_change('value', callback_mode)

# Inicializar modo mixto visible
line_renderer.visible = True
points_renderer.visible = True
points_renderer.glyph.fill_color = 'blue'
p.toolbar.active_drag = None

# Sincronizar datos fuente_all con line y points para modo editable
sync_callback = CustomJS(args=dict(source_all=source_all, source_line=source_line, source_points=source_points), code="""
    // Actualizar line y points desde source_all datos
    const d_all = source_all.data;
    source_line.data = {x: d_all['x'], y: d_all['y']};
    source_points.data = {x: d_all['x'], y: d_all['y']};
    source_line.change.emit();
    source_points.change.emit();
""")

# Cada vez que source_all cambia (por mover puntos), sincronizar
source_all.js_on_change('data', sync_callback)

# Cuando se mueven puntos en modo editable, actualizar source_all
sync_points_callback = CustomJS(args=dict(source_all=source_all, source_points=source_points), code="""
    const d_all = source_all.data;
    const d_points = source_points.data;
    for (let i = 0; i < d_points['x'].length; i++) {
        d_all['x'][i] = d_points['x'][i];
        d_all['y'][i] = d_points['y'][i];
    }
    source_all.change.emit();
""")

source_points.js_on_change('data', sync_points_callback)

layout = column(select_mode, p)
show(layout)


In [6]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, PointDrawTool, CustomJS, Button
from bokeh.layouts import column

output_notebook()

# Datos iniciales
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

source = ColumnDataSource(data=dict(x=x, y=y))

p = figure(title="Mover y eliminar puntos con Bokeh + CustomJS",
           x_range=(0,6), y_range=(0,8), tools="tap")

# Puntos
renderer = p.scatter('x', 'y', source=source, size=10, color="blue")

# Línea que une puntos
line_renderer = p.line('x', 'y', source=source, line_width=2, color="green")

# Herramienta para mover puntos
draw_tool = PointDrawTool(renderers=[renderer])
p.add_tools(draw_tool)
p.toolbar.active_drag = draw_tool

# Callback JS para actualizar línea cuando cambian los puntos
callback = CustomJS(args=dict(source=source, line=line_renderer), code="""
    const data = source.data;
    line.data_source.data['x'] = data['x'];
    line.data_source.data['y'] = data['y'];
    line.data_source.change.emit();
""")
source.js_on_change('data', callback)

# Botón eliminar punto seleccionado
btn_eliminar = Button(label="Eliminar punto seleccionado", button_type="danger", width=200)

btn_eliminar.js_on_click(CustomJS(args=dict(source=source), code="""
    const inds = source.selected.indices;
    if (inds.length == 0) {
        alert('Selecciona al menos un punto para eliminar');
        return;
    }
    const data = source.data;
    inds.sort((a,b) => b - a);  // eliminar de atrás para no cambiar índices
    for (let i of inds) {
        data['x'].splice(i, 1);
        data['y'].splice(i, 1);
    }
    source.selected.indices = [];
    source.change.emit();
"""))

layout = column(p, btn_eliminar)
show(layout)


In [ ]:
# No funciona directamente en Colab sin configuración adicional, pero si usas local sí
# Aquí te doy un ejemplo básico que puedes correr en tu PC local:

import dash
from dash import dcc, html, Input, Output, State, dash_table
import plotly.graph_objs as go
import pandas as pd

app = dash.Dash(__name__)

df = pd.DataFrame({
    'X': [1,2,3,4,5],
    'Y': [2,4,1,3,7]
})

app.layout = html.Div([
    dash_table.DataTable(
        id='editable-table',
        columns=[{'name': c, 'id': c, 'editable': True} for c in df.columns],
        data=df.to_dict('records')
    ),
    dcc.Graph(id='graph')
])

@app.callback(
    Output('graph', 'figure'),
    Input('editable-table', 'data')
)
def update_graph(rows):
    dff = pd.DataFrame(rows)
    fig = go.Figure(data=go.Scatter(x=dff['X'], y=dff['Y'], mode='lines+markers'))
    return fig

if __name__ == '__main__':
    app.run(debug=True)



<IPython.core.display.Javascript object>

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
import numpy as np

output_notebook()

# Datos de ejemplo
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"]*len(x)))

# Crear figura
p = figure(title="Gráfica interactiva con distintos tipos", tools="tap,box_select,lasso_select,reset", width=800, height=400)

# Glyphs: puntos y línea
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# Selector de tipo de gráfica
selector_tipo = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])

# Selector de color general
selector_color = Select(title="Color general:", value="blue", options=["blue", "red", "green", "orange", "black", "purple"])

# Callback para cambiar visibilidad de glyphs
callback_tipo = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;
    puntos.visible = (tipo !== "línea");
    linea.visible = (tipo !== "puntos");
""")
selector_tipo.js_on_change('value', callback_tipo)

# Callback para cambiar color general
callback_color = CustomJS(args=dict(source=source), code="""
    const color = cb_obj.value;
    const data = source.data;

    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = color;
    }

    source.change.emit();
""")
selector_color.js_on_change('value', callback_color)

# Callback para colorear puntos seleccionados
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;

    // Guardar color actual (el que está en el primer punto no seleccionado)
    let base_color = "blue";
    for (let i = 0; i < data['color'].length; i++) {
        if (!indices.includes(i)) {
            base_color = data['color'][i];
            break;
        }
    }

    // Restaurar todos al color base
    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = base_color;
    }

    // Colorear seleccionados de rojo
    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }

    source.change.emit();
"""))

# Tabla de datos
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# Layout completo
layout = column(selector_tipo, selector_color, row(p, tabla))
show(layout)


/tmp/ipython-input-4089284841.py:17: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.



In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, DataTable, TableColumn
from bokeh.layouts import column, row
import numpy as np

output_notebook()

# Datos de ejemplo
x = np.linspace(0, 10, 50)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y, color=["blue"]*len(x)))

# Crear figura con rangos fijos
p = figure(
    title="Gráfica interactiva con distintos tipos",
    tools="tap,box_select,lasso_select,reset",
    width=800,
    height=400,
    x_range=(0, 10),
    y_range=(-1.5, 1.5)  # Rango fijo evita que el canvas se mueva
)

# Glyphs: puntos y línea
puntos = p.circle('x', 'y', source=source, size=8, fill_color='color', line_color=None)
linea = p.line('x', 'y', source=source, line_width=2, color='blue')

# Selector de tipo de gráfica
selector = Select(title="Tipo de gráfica:", value="puntos", options=["puntos", "línea", "mixto"])

# Callback JS para cambiar visibilidad según selección
callback = CustomJS(args=dict(puntos=puntos, linea=linea), code="""
    const tipo = cb_obj.value;

    if (tipo === "puntos") {
        puntos.visible = true;
        linea.visible = false;
    } else if (tipo === "línea") {
        puntos.visible = false;
        linea.visible = true;
    } else if (tipo === "mixto") {
        puntos.visible = true;
        linea.visible = true;
    }
""")

selector.js_on_change('value', callback)

# Cambiar color al seleccionar puntos
source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    const data = source.data;

    for (let i = 0; i < data['color'].length; i++) {
        data['color'][i] = "blue";
    }

    for (let i = 0; i < indices.length; i++) {
        data['color'][indices[i]] = "red";
    }

    source.change.emit();
"""))

# Tabla para mostrar datos
columns = [TableColumn(field="x", title="X"), TableColumn(field="y", title="Y")]
tabla = DataTable(source=source, columns=columns, width=400, height=300)

# Layout y mostrar
layout = column(selector, row(p, tabla))

out = widgets.Output(layout=widgets.Layout(border='1px solid gray', height='500px'))

with out:
    show(layout)

display(out)

Output(layout=Layout(border='1px solid gray', height='500px'))

In [ ]:
import ipywidgets as widgets
from IPython.display import display

out = widgets.Output(layout=widgets.Layout(border='1px solid gray', height='500px'))

with out:
    show(layout)

display(out)


Output(layout=Layout(border='1px solid gray', height='500px'))

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, DataTable, TableColumn, NumberFormatter
from bokeh.layouts import column
import pandas as pd
import numpy as np

output_notebook()

# === Simular datos (nivel del mar por hora) ===
tiempo = np.arange(1, 25)  # 24 horas
np.random.seed(42)
nivel_mar = np.random.normal(loc=100, scale=10, size=24)

# === Crear DataFrame profesional ===
df = pd.DataFrame({
    'Hora': tiempo,
    'Nivel del Mar': np.round(nivel_mar, 2),
    'Promedio': np.round([np.mean(nivel_mar[:i+1]) for i in range(24)], 2)
})

# === Fuente para Bokeh ===
source = ColumnDataSource(df)

# === Tabla profesional con formato ===
columns = [
    TableColumn(field='Hora', title='Hora', formatter=NumberFormatter(format="0")),
    TableColumn(field='Nivel del Mar', title='Nivel del Mar (cm)', formatter=NumberFormatter(format="0.00")),
    TableColumn(field='Promedio', title='Promedio Acumulado', formatter=NumberFormatter(format="0.00"))
]

data_table = DataTable(source=source, columns=columns, width=600, height=280, index_position=None)

# === Gráfica sincronizada ===
p = figure(title="Nivel del Mar por Hora", x_axis_label="Hora", y_axis_label="Nivel del Mar (cm)", width=700, height=350)
p.line(x='Hora', y='Nivel del Mar', source=source, line_width=2)
p.circle(x='Hora', y='Nivel del Mar', source=source, size=6, color="navy", alpha=0.5)

# === Mostrar todo junto ===
show(column(p, data_table))


In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, DataTable, TableColumn, CustomJS, Circle, CDSView, IndexFilter
from bokeh.layouts import column
import pandas as pd
import numpy as np

output_notebook()

# === Datos simulados ===
tiempo = np.arange(20)
datos = np.random.randint(10, 100, size=20)

df = pd.DataFrame({
    'Tiempo': tiempo,
    'Nivel': datos
})

source = ColumnDataSource(df)
highlight_source = ColumnDataSource(data={k: [] for k in df.columns})  # Para resaltar punto seleccionado

# === Tabla Interactiva ===
columns = [
    TableColumn(field="Tiempo", title="Tiempo"),
    TableColumn(field="Nivel", title="Nivel")
]
data_table = DataTable(source=source, columns=columns, width=300, height=280)

# === Gráfica ===
p = figure(title="Nivel vs Tiempo", width=600, height=300, tools="tap", tooltips=[("Tiempo", "@Tiempo"), ("Nivel", "@Nivel")])
p.line('Tiempo', 'Nivel', source=source, line_width=2)
p.circle('Tiempo', 'Nivel', size=8, source=source)

# Círculo de resaltado
p.circle('Tiempo', 'Nivel', size=12, fill_color='red', line_color='black', source=highlight_source)

# === Sincronización Tabla → Gráfica ===
source.selected.js_on_change("indices", CustomJS(args=dict(
    source=source,
    highlight_source=highlight_source
), code="""
    const selected_idx = cb_obj.indices[0];
    if (selected_idx === undefined) {
        highlight_source.data = { Tiempo: [], Nivel: [] };
    } else {
        const tiempo = source.data['Tiempo'][selected_idx];
        const nivel = source.data['Nivel'][selected_idx];
        highlight_source.data = { Tiempo: [tiempo], Nivel: [nivel] };
    }
    highlight_source.change.emit();
"""))

# === Mostrar todo ===
show(column(p, data_table))


In [ ]:
# Instalar ipywidgets (si no lo tienes)
!pip install ipywidgets

# Importar widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Crear elementos
entrada = widgets.Textarea(
    placeholder='Escribe tu sugerencia o duda aquí...',
    description='Mensaje:',
    layout=widgets.Layout(width='100%', height='100px')
)

boton = widgets.Button(
    description='Enviar',
    button_style='success'
)

salida = widgets.Output()

# Lista para guardar mensajes
mensajes = []

# Función al hacer clic
def enviar_mensaje(b):
    mensaje = entrada.value.strip()
    if mensaje:
        mensajes.append(mensaje)
        entrada.value = ""  # limpiar entrada
        with salida:
            clear_output()
            for i, msg in enumerate(mensajes, 1):
                print(f"{i}. {msg}")

# Conectar botón
boton.on_click(enviar_mensaje)

# Mostrar en Colab
display(entrada, boton, salida)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00


Textarea(value='', description='Mensaje:', layout=Layout(height='100px', width='100%'), placeholder='Escribe t…

Button(button_style='success', description='Enviar', style=ButtonStyle())

Output()

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, CheckboxGroup
from bokeh.layouts import column
import numpy as np

output_notebook()

# === Datos de ejemplo ===
x = np.arange(0, 10, 0.5)
y = np.sin(x)
source2 = ColumnDataSource(data=dict(x=x, y=y))

# === Figura ===
p = figure(title="Haz clic o selecciona un rango", height=300, width=600, tools="box_select,lasso_select,reset")

# Línea con círculos
p.line('x', 'y', source=source2)
p.circle('x', 'y', source=source2, size=8, selection_color="firebrick", nonselection_alpha=0.1)

# === Checkbox para activar enfoque ===
checkbox = CheckboxGroup(labels=["Enfocar selección"], active=[])

# === Callback para ajustar rangos al seleccionar puntos ===
restore_callback = CustomJS(args=dict(source=source2, plot=p, checkbox=checkbox), code="""
    if (checkbox.active.includes(0)) {
        const indices = source.selected.indices;
        if (indices.length === 0) return;

        const data = source.data;
        let x_vals = indices.map(i => data['x'][i]);
        let y_vals = indices.map(i => data['y'][i]);

        const x_min = Math.min(...x_vals);
        const x_max = Math.max(...x_vals);
        const y_min = Math.min(...y_vals);
        const y_max = Math.max(...y_vals);

        // Ajustar los rangos con un pequeño margen
        const margen = 0.1;
        plot.x_range.start = x_min - margen;
        plot.x_range.end = x_max + margen;
        plot.y_range.start = y_min - margen;
        plot.y_range.end = y_max + margen;
    }
""")

# Asociar el evento de selección
source2.selected.js_on_change('indices', restore_callback)

# Mostrar
show(column(checkbox, p))


Support for third party widgets will remain active for the duration of the session. To disable support: